# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  Dec 19 2022 10:08AM  253357        10  Clear-8922281   
1  Dec 19 2022 10:03AM  253346        15       30012924   
2  Dec 19 2022 10:03AM  253346        15       30012925   
3  Dec 19 2022 10:03AM  253346        15       30012926   
4  Dec 19 2022 10:03AM  253346        15       30012927   
5  Dec 19 2022 10:03AM  253346        15       30012928   
6  Dec 19 2022 10:03AM  253346        15       30012930   
7  Dec 19 2022 10:03AM  253346        15       30012932   
8  Dec 19 2022 10:03AM  253346        15       30012933   
9  Dec 19 2022 10:03AM  253346        15       30012935   

                Customer ShipmentStatus  
0         ClearSpec, LLC       Released  
1  Alliance Pharma, Inc.       Released  
2  Alliance Pharma, Inc.       Released  
3  Alliance Pharma, Inc.       Released  
4  Alliance Pharma, Inc.       Released  
5  Alliance Pharma, Inc.       Released  
6  Alliance Pharma, Inc.       Released  
7  Alliance Pharma, Inc.       Released  
8  Alliance Pharma, Inc.       Released  
9  Alliance Pharma, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
8   253344       Released          6
9   253345      Executing          2
10  253345       Released          3
11  253346       Released         35
12  253357       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
253343                NaN       1.0
253344                NaN       6.0
253345                2.0       3.0
253346                NaN      35.0
253357                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
253332                1.0      12.0
253335                0.0      28.0
253337                1.0       0.0
253340                0.0      19.0
253341                0.0      39.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
253332                  1        12
253335                  0        28
253337                  1         0
253340                  0        19
253341                  0        39

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               253332          1        12
1               253335          0        28
2               253337          1         0
3               253340          0        19
4               253341          0        39

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               253332         1       12
1               253335                 28
2               253337         1         
3               253340                 19
4               253341                 39

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                            Customer
0    Dec 19 2022 10:08AM  253357        10                      ClearSpec, LLC
1    Dec 19 2022 10:03AM  253346        15               Alliance Pharma, Inc.
36   Dec 19 2022  9:45AM  253345        10                    Eywa Pharma Inc.
41   Dec 19 2022  9:44AM  253344        10                             Bio-PRF
47   Dec 19 2022  9:37AM  253343        16      Sartorius Bioprocess Solutions
48   Dec 19 2022  9:36AM  253340        10                   ISDIN Corporation
67   Dec 19 2022  9:35AM  253342        16  Sartorius Lab Products and Service
68   Dec 19 2022  9:33AM  253341        10                    Methapharm, Inc.
107  Dec 19 2022  9:11AM  253337        16                Emersa Waterbox, LLC
108  Dec 19 2022  8:48AM  253335        12                           Hush Hush

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                            Customer  \
0  Dec 19 2022 10:08AM  253357        10                      ClearSpec, LLC   
1  Dec 19 2022 10:03AM  253346        15               Alliance Pharma, Inc.   
2  Dec 19 2022  9:45AM  253345        10                    Eywa Pharma Inc.   
3  Dec 19 2022  9:44AM  253344        10                             Bio-PRF   
4  Dec 19 2022  9:37AM  253343        16      Sartorius Bioprocess Solutions   
5  Dec 19 2022  9:36AM  253340        10                   ISDIN Corporation   
6  Dec 19 2022  9:35AM  253342        16  Sartorius Lab Products and Service   
7  Dec 19 2022  9:33AM  253341        10                    Methapharm, Inc.   
8  Dec 19 2022  9:11AM  253337        16                Emersa Waterbox, LLC   
9  Dec 19 2022  8:48AM  253335        12                           Hush Hush   

  Executing Released  
0                  1  
1                 35  
2         2        3  
3                  6  
4                  1  
5                 19  
6                  1  
7                 39  
8         1           
9                 28

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                            Customer  \
0  Dec 19 2022 10:08AM  253357        10                      ClearSpec, LLC   
1  Dec 19 2022 10:03AM  253346        15               Alliance Pharma, Inc.   
2  Dec 19 2022  9:45AM  253345        10                    Eywa Pharma Inc.   
3  Dec 19 2022  9:44AM  253344        10                             Bio-PRF   
4  Dec 19 2022  9:37AM  253343        16      Sartorius Bioprocess Solutions   
5  Dec 19 2022  9:36AM  253340        10                   ISDIN Corporation   
6  Dec 19 2022  9:35AM  253342        16  Sartorius Lab Products and Service   
7  Dec 19 2022  9:33AM  253341        10                    Methapharm, Inc.   
8  Dec 19 2022  9:11AM  253337        16                Emersa Waterbox, LLC   
9  Dec 19 2022  8:48AM  253335        12                           Hush Hush   

  Released Executing  
0        1            
1       35            
2        3         2  
3        6            
4        1            
5       19            
6        1            
7       39            
8                  1  
9       28

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Dec 19 2022 10:08AM  253357        10                  ClearSpec, LLC   
1  Dec 19 2022 10:03AM  253346        15           Alliance Pharma, Inc.   
2  Dec 19 2022  9:45AM  253345        10                Eywa Pharma Inc.   
3  Dec 19 2022  9:44AM  253344        10                         Bio-PRF   
4  Dec 19 2022  9:37AM  253343        16  Sartorius Bioprocess Solutions   

  Released Executing  
0        1            
1       35            
2        3         2  
3        6            
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                        Customer  \
0  Dec 19 2022 10:08AM  253357        10                  ClearSpec, LLC   
1  Dec 19 2022 10:03AM  253346        15           Alliance Pharma, Inc.   
2  Dec 19 2022  9:45AM  253345        10                Eywa Pharma Inc.   
3  Dec 19 2022  9:44AM  253344        10                         Bio-PRF   
4  Dec 19 2022  9:37AM  253343        16  Sartorius Bioprocess Solutions   

   Released  Executing  
0       1.0        NaN  
1      35.0        NaN  
2       3.0        2.0  
3       6.0        NaN  
4       1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Dec 19 2022 10:08AM  253357        10                  ClearSpec, LLC   
1  Dec 19 2022 10:03AM  253346        15           Alliance Pharma, Inc.   
2  Dec 19 2022  9:45AM  253345        10                Eywa Pharma Inc.   
3  Dec 19 2022  9:44AM  253344        10                         Bio-PRF   
4  Dec 19 2022  9:37AM  253343        16  Sartorius Bioprocess Solutions   

   Released  Executing  
0       1.0        0.0  
1      35.0        0.0  
2       3.0        2.0  
3       6.0        0.0  
4       1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1266727      68.0        2.0
12          253335      28.0        0.0
15          253346      35.0        0.0
16          760022       2.0        1.0
19          253332      12.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1266727      68.0        2.0
1        12   253335      28.0        0.0
2        15   253346      35.0        0.0
3        16   760022       2.0        1.0
4        19   253332      12.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      68.0        2.0
1        12      28.0        0.0
2        15      35.0        0.0
3        16       2.0        1.0
4        19      12.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   68.0
1        12  Released   28.0
2        15  Released   35.0
3        16  Released    2.0
4        19  Released   12.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12    15   16    19
Status                                
Executing   2.0   0.0   0.0  1.0   1.0
Released   68.0  28.0  35.0  2.0  12.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12    15   16    19
0          Executing   2.0   0.0   0.0  1.0   1.0
1           Released  68.0  28.0  35.0  2.0  12.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12    15   16    19
0  Executing   2.0   0.0   0.0  1.0   1.0
1   Released  68.0  28.0  35.0  2.0  12.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()